In [30]:
import os
import re
import requests
import datetime as dt
import numpy as np
import pandas as pd


In [16]:
data_url = "/Users/RichardAfolabi/myGitHub/_datasets_downloads/malware/Malware_Passive_DNS_Data/"
cols_names = ['Date, MD5Hash', 'Domain', 'IP_Address']

### Read all pieces of `36 data files` into pandas dataframe and merge them all together into one dataframe

In [41]:
pieces = []

# Create numbering for list of months = [01,12] to match the format of the input files
file_mnths = [str(0)+str(mnth) if mnth<10 else str(mnth) for mnth in range(1,13)]

# Loop over months for each year = [2011, 2012, 2013]. Read and merge the data files into lists.
for yr in np.arange(2011,2014):
    for mnth in file_mnths:
        path = data_url+'gt_mpdns_'+str(yr)+'-'+mnth+'.csv'
        malware = pd.read_csv(path, names=cols_names, parse_dates=True, header=None, index_col=0)
        malware['Year'] = yr
        malware['month'] = mnth
        pieces.append(malware)

# Concatenate the list of files into single dataframe
malware_dataset = pd.concat(pieces)  

parsed_date = pd.to_datetime(malware_dataset.index.values)
malware_dataset['elapsedInDays'] = (parsed_date - pd.datetime(2011,1,1)).days

malware_dataset.head()


,"Date, MD5Hash",Domain,IP_Address,Year,month,elapsedInDays
2011-01-01,0003b97144cf06801627fdd07a917f51,mozilla.com,63.245.209.106,2011,01,0
2011-01-01,0003b97144cf06801627fdd07a917f51,rapidshare.com,195.122.131.12,2011,01,0
2011-01-01,0003b97144cf06801627fdd07a917f51,tagshare.in,69.10.39.21,2011,01,0
2011-01-01,00134ecf2477c1f477a8767c87572107,mozilla.com,63.245.209.106,2011,01,0
2011-01-01,00134ecf2477c1f477a8767c87572107,rapidshare.com,195.122.131.5,2011,01,0


#### Save merged dataset to single `CVS`. Check length of dataset. Print  first 5 records in the merged dataset. 

* Waiting time : 15mins 
* Datafile size: 3.5GB 

In [ ]:

malware_dataset.to_csv("~/myGitHub/_datasets_downloads/malware/malware_dataset.csv")

print("\n *** Length of dataset: {:,} data rows".format(malware_dataset.index.size))
print("\n ::: Size of merged data: {:,} bytes ::: ".format(os.path.getsize('malware_dataset.csv')))
malware_dataset.head(2)



## `Data Cleaning`
Some rows may have `NaN` entries. We should decide how to deal with them. The main column is the `IP_Address` column without which we cannot do much. So if `IP Address` is missing or Null, we exclude the row from our dataset. However, since we can reverse engineer the `IP` from the `Domains` column using online `Domain LookUp` tools, we can use only the clean data with IPs and then reserve `reverse_engineered_ips` for `Machine Learning` evaluation.

In [ ]:
# Rows with IPs we can use now for development
clean_malware_dataset = malware_dataset[malware_dataset.IP_Address.notnull()]
clean_malware_dataset.to_csv("~/myGitHub/_datasets_downloads/malware/clean_malware_dataset.csv")

# Rows with missing IP we can reverse-engineer to find IP and use for machine learning
missing_ip = malware_dataset[malware_dataset.IP_Address.isnull()]
ratiox = (missing_ip.index.size/malware_dataset.index.size)*100

# Dataset with missing IP and only domain attribute present
missing_ip.to_csv("~/myGitHub/_datasets_downloads/malware/missing_ip.csv")

print("\nRows with missing IPs : {:,} \t => {:.1f}% of whole data".format(missing_ip.index.size, ratiox))
print("\nRows with missing Domains : {:,} ".format(malware_dataset[malware_dataset.Domains.isnull()].size))



#### Create function to compute `elapsedInDays` and `elapsedInWks`

In [ ]:
def elapsedInDaysFrom(df,yyyy,mm,dd):
    """ Returns the elapsed date from date specified. """
    
    parsed_date = pd.to_datetime(df.index.values)
    result = df.copy()
    
    result['elapsedInDays'] = (parsed_date - pd.datetime(yyyy,mm,dd)).days
    result['elapsedInWks'] = ((result.elapsedInDays)/7).round()
    return result


# Call function to create `elapsedInDays` and `elapsedInWks` from the 1st date
malware_data = elapsedInDaysFrom(clean_malware_dataset, 2011,1,1)
malware_data.tail(2)



# Import datasets with missing IP

In [2]:
missing_ip = pd.read_csv("~/myGitHub/_datasets_downloads/malware/missing_ip.csv")
missing_ip.head()

,Unnamed: 0,Date,MD5Hash,Domains,IP_Address,Year,month
0,21,2011-01-01,01079c99114b0e8edfc070052afab0c5,antispyware-kit.com,NaN,2011,1
1,22,2011-01-01,01079c99114b0e8edfc070052afab0c5,antispyware-kit.net,NaN,2011,1
2,60,2011-01-01,02b34d6ff38c05462144ffb3eb030239,xibudific.cn,NaN,2011,1
3,63,2011-01-01,02e2837cc9880dd9d187c182175cf130,antispyware-kit.com,NaN,2011,1
4,64,2011-01-01,02e2837cc9880dd9d187c182175cf130,antispyware-kit.net,NaN,2011,1


# Get the unique domains

In [3]:
unique_domains = pd.DataFrame(missing_ip.Domains.unique())
unique_domains.columns = ['Domains']
print("\n Unique Domains : {:,}".format(unique_domains.index.size ))
unique_domains.tail()


 Unique Domains : 196,468


,Domains
196463,karemzo.no-ip.biz
196464,rolex211.8s.nl
196465,rolex212.8s.nl
196466,rolex213.8s.nl
196467,catch-cdn.com


# Function to extract geolocation data using freegeoips API

In [ ]:

fetched_data = []

def lookup_domains(list_of_domains):
    for domain in list_of_domains:
        r = requests.get("http://freegeoip.net/json/"+domain)
        if r.ok and r.status_code != '404':
            fetched_data.append(r.json())
    return pd.DataFrame(fetched_data)


domain_data = lookup_domains(unique_domains.Domains.values)


In [ ]:
print("\n\n Number of rows: {:,}".format(domain_data.index.size))
domain_data.head()